# 4. RNN Model All

* RNN model using the entire dataset (not separated into clusters)

# imports

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm
from datetime import datetime
from IPython.display import display
import tensorflow
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from numpy import array
from numpy import hstack
from tqdm.keras import TqdmCallback

# pd.display options

In [21]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

# Loading the data

In [22]:
df_train = pd.read_pickle("../Data/Prepared_Data/supervised/consumption_all.pkl")
df_train

,meter_id,meter_id_0,meter_id_1,meter_id_2,meter_id_3,meter_id_4,meter_id_5,meter_id_6,meter_id_7,meter_id_8,meter_id_9,meter_id_10,meter_id_11,meter_id_12,date,day_of_year_sin,day_of_year_cos,day_of_week,day_of_month,month_ord,month_sin,month_cos,is_weekend,energy_cluster,num_bedrooms,detached,flat,semi_detached,terraced,energy n-7,energy n-6,energy n-5,energy n-4,energy n-3,energy n-2,energy n-1,meter_reading
0,0x240e5e22734a44a174b7dabcf1ea00d70d9ec168,0,0,0,0,0,0,0,0,0,0,0,0,1,2017-12-31,1.726064e-02,0.999851,6,31,12,0.540641,0.841254,1,1,3.0,0.0,0.0,1.0,0.0,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495
1,0x2ce3d582a1316db5bcfe405cbd6070268944778e,0,0,0,0,0,0,0,0,0,0,0,1,0,2017-12-31,1.726064e-02,0.999851,6,31,12,0.540641,0.841254,1,1,3.0,0.0,0.0,1.0,0.0,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495
2,0x306e6baa9367d3c43fa6ecc2d0054b207d6ef471,0,0,0,0,0,0,0,0,0,0,0,1,1,2017-12-31,1.726064e-02,0.999851,6,31,12,0.540641,0.841254,1,1,3.0,0.0,0.0,1.0,0.0,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495
3,0x391e6c2169c27de797ccbdf2d623365da28a6d3d,0,0,0,0,0,0,0,0,0,0,1,0,0,2017-12-31,1.726064e-02,0.999851,6,31,12,0.540641,0.841254,1,1,3.0,0.0,0.0,1.0,0.0,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495
4,0x423fa805ddb0cba9bdb4460f9a78540287eefd0e,0,0,0,0,0,0,0,0,0,0,1,0,1,2017-12-31,1.726064e-02,0.999851,6,31,12,0.540641,0.841254,1,1,3.0,0.0,0.0,1.0,0.0,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617640,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,1,0,0,0,0,2017-12-27,-5.176135e-02,0.998659,2,27,12,0.540641,0.841254,0,0,3.0,1.0,0.0,0.0,0.0,21.999000,26.232000,14.036000,15.990000,15.550500,15.111000,16.506000,18.758500
617641,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,1,0,0,0,0,2017-12-28,-3.451614e-02,0.999404,3,28,12,0.540641,0.841254,0,0,3.0,1.0,0.0,0.0,0.0,26.232000,14.036000,15.990000,15.550500,15.111000,16.506000,18.758500,21.011000
617642,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,1,0,0,0,0,2017-12-29,-1.726064e-02,0.999851,4,29,12,0.540641,0.841254,0,0,3.0,1.0,0.0,0.0,0.0,14.036000,15.990000,15.550500,15.111000,16.506000,18.758500,21.011000,19.204000
617643,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,1,0,0,0,0,2017-12-30,-2.449294e-16,1.000000,5,30,12,0.540641,0.841254,1,0,3.0,1.0,0.0,0.0,0.0,15.990000,15.550500,15.111000,16.506000,18.758500,21.011000,19.204000,17.397000


# RNN Model
## Converting to 3 Dimensional so it can be understood by RNN based models
transform input from [samples, features] to [samples, timesteps, features]

In [23]:
### various feature combinations we may want to use ###

# no addInfo and no day_of_year and no energy_cluster
# so contains meter_id's, day of year, day of month,  cyclically encoded month and is_weekend 
dropped_features = ["num_bedrooms","detached","flat","semi_detached","terraced","day_of_year_sin","day_of_year_cos","energy_cluster","month_ord"] 

In [24]:
#converting unclustered into 3 dimensions
X = df_train.drop({"meter_reading","meter_id","date"}|set(dropped_features),axis=1)
y = df_train["meter_reading"] 

print("X shape",X.shape,"| y shape", y.shape)

display(X)
display(y)

#converting features and labels into np array
X = X.values
y = y.values

X shape (617645, 25) | y shape (617645,)


,meter_id_0,meter_id_1,meter_id_2,meter_id_3,meter_id_4,meter_id_5,meter_id_6,meter_id_7,meter_id_8,meter_id_9,meter_id_10,meter_id_11,meter_id_12,day_of_week,day_of_month,month_sin,month_cos,is_weekend,energy n-7,energy n-6,energy n-5,energy n-4,energy n-3,energy n-2,energy n-1
0,0,0,0,0,0,0,0,0,0,0,0,0,1,6,31,0.540641,0.841254,1,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536
1,0,0,0,0,0,0,0,0,0,0,0,1,0,6,31,0.540641,0.841254,1,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536
2,0,0,0,0,0,0,0,0,0,0,0,1,1,6,31,0.540641,0.841254,1,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536
3,0,0,0,0,0,0,0,0,0,0,1,0,0,6,31,0.540641,0.841254,1,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536
4,0,0,0,0,0,0,0,0,0,0,1,0,1,6,31,0.540641,0.841254,1,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617640,0,1,1,0,0,1,0,1,1,0,0,0,0,2,27,0.540641,0.841254,0,21.999000,26.232000,14.036000,15.990000,15.550500,15.111000,16.506000
617641,0,1,1,0,0,1,0,1,1,0,0,0,0,3,28,0.540641,0.841254,0,26.232000,14.036000,15.990000,15.550500,15.111000,16.506000,18.758500
617642,0,1,1,0,0,1,0,1,1,0,0,0,0,4,29,0.540641,0.841254,0,14.036000,15.990000,15.550500,15.111000,16.506000,18.758500,21.011000
617643,0,1,1,0,0,1,0,1,1,0,0,0,0,5,30,0.540641,0.841254,1,15.990000,15.550500,15.111000,16.506000,18.758500,21.011000,19.204000


0         11.881495
1         11.881495
2         11.881495
3         11.881495
4         11.881495
            ...    
617640    18.758500
617641    21.011000
617642    19.204000
617643    17.397000
617644    15.237000
Name: meter_reading, Length: 617645, dtype: float64

In [25]:
#transforming input from [samples, features] to [samples, timesteps, features] as expected by RNN
X = X.reshape((X.shape[0], X.shape[1], 1))
print("X shape",X.shape)

X shape (617645, 25, 1)


In [26]:
#changing type of array so tensorflow can understand
#X = np.asarray(X).astype('float32')
y = np.asarray(y).astype('float32')





## Defining the RNN model




In [27]:
#parameters
n_features = X.shape[2]
n_steps_in = 7 # using the last 7 inputs
n_steps_out = 1 # 1 output

In [28]:
#defining the model
#2 LSTM layers, Dense output layer
model = Sequential()
model.add(SimpleRNN(100, activation='relu', return_sequences=True, input_shape=[None, n_features])) #None or n_steps_in
model.add(SimpleRNN(25, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')

In [29]:
# fit model
model.fit(X, y, epochs=1, verbose=0, callbacks=[TqdmCallback(verbose=2)])

0epoch [00:00, ?epoch/s]

  0%|          | 0.00/19.3k [00:00<?, ?batch/s]

## predicting from the model

In [30]:
## preds ##

# loading the predictions dataset
df_preds = pd.read_pickle("../Data/Prepared_Data/supervised/consumption_all_preds.pkl")

# splitting by meter id
df_meters = {k: v for (k, v) in df_preds.groupby('meter_id')}

meter_ids = df_preds.meter_id.unique()

X_preds = {}
y_preds = {}

# splitting each of these into X and y
for i in tqdm(range(len(df_meters))):
    meter_id = meter_ids[i]
    
    #getting X and y for this meter
    X_preds[meter_id] = (df_preds.loc[df_preds["meter_id"]==meter_id].drop({"meter_reading","meter_id","date"}|set(dropped_features),axis=1))
    y_preds[meter_id] = (df_preds.loc[df_preds["meter_id"]==meter_id]["meter_reading"] )
    #converting features and labels into np array
    X_preds[meter_id] = X_preds[meter_id].values
    y_preds[meter_id] = y_preds[meter_id].values
    #transforming input from [samples, features] to [samples, timesteps, features] as expected by RNN
    X_preds[meter_id] = X_preds[meter_id].reshape((X_preds[meter_id].shape[0], X_preds[meter_id].shape[1], 1))


  0%|          | 0/3248 [00:00<?, ?it/s]

In [31]:
## train ##

# splitting by meter id
df_meters = {k: v for (k, v) in df_train.groupby('meter_id')}

meter_ids = df_preds.meter_id.unique()

X_train = {}
y_train = {}

# splitting each of these into X and y
for i in tqdm(range(len(df_meters))):
    meter_id = meter_ids[i]
    
    #getting X and y for this meter
    X_train[meter_id] = (df_preds.loc[df_preds["meter_id"]==meter_id].drop({"meter_reading","meter_id","date"}|set(dropped_features),axis=1))
    y_preds[meter_id] = (df_preds.loc[df_preds["meter_id"]==meter_id]["meter_reading"] )
    #converting features and labels into np array
    X_train[meter_id] = X_train[meter_id].values
    y_preds[meter_id] = y_preds[meter_id].values
    #transforming input from [samples, features] to [samples, timesteps, features] as expected by RNN
    X_preds[meter_id] = X_preds[meter_id].reshape((X_preds[meter_id].shape[0], X_preds[meter_id].shape[1], 1))

  0%|          | 0/3248 [00:00<?, ?it/s]

In [32]:

#making the dataframe to store the forecasts
rangeOfDates = pd.date_range(datetime(2018,1,1),datetime(2018,12,31),freq="D")
df_forecasts = pd.DataFrame(columns=["meter_id"],data=meter_ids)
for new_sample in tqdm(rangeOfDates):
    df_forecasts[str(new_sample.date())]=np.nan
display(df_forecasts)


  0%|          | 0/365 [00:00<?, ?it/s]

,meter_id,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08,2018-01-09,2018-01-10,2018-01-11,2018-01-12,2018-01-13,2018-01-14,2018-01-15,2018-01-16,2018-01-17,2018-01-18,2018-01-19,2018-01-20,2018-01-21,2018-01-22,2018-01-23,2018-01-24,...,2018-12-07,2018-12-08,2018-12-09,2018-12-10,2018-12-11,2018-12-12,2018-12-13,2018-12-14,2018-12-15,2018-12-16,2018-12-17,2018-12-18,2018-12-19,2018-12-20,2018-12-21,2018-12-22,2018-12-23,2018-12-24,2018-12-25,2018-12-26,2018-12-27,2018-12-28,2018-12-29,2018-12-30,2018-12-31
0,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0x459c834d1f6cfb5b734b82aa9f5410fa97fb70da,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0x4a1ed36825360a058cec2bdd409fc2459e1ce54f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0x5b76d3c0e0aefc6e0a8d1d031f96388a23263407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0x943ebe39ef2be6ef807c42c5a647e27112ca5b0f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3243,0x7dd7a7b8ee1bec7c44b24f738c752482f6161065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3244,0xfdaf9f857621ec06f2cf801f42a020a322835090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3245,0xd28f2f001e0cd4d6c121a3cb2e1427207e170e18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3246,0x47218b46abb2fcaade487a211911406dc6e13730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
### making predictions iteratively ###

def make_preds ():
    # for each meter
    for i in tqdm(range(len(meter_ids))):
        meter_id = meter_ids[i]
        meter_X_train = X_train[meter_id]
        meter_X_preds = X_preds[meter_id]

        meter_X = meter_X_train
        

        
        # predicting the next 365 days
        j=0
        for this_date in tqdm(rangeOfDates, leave=False):
            date_name = str(this_date.date())
            
            y_pred_one = model.predict(meter_X[-7:,:])[:, np.newaxis, :] # making prediction based on the last 7 days
            
            print(meter_id, date_name,y_pred_one.shape,y_pred_one[0][0][0])
            print()
            df_forecasts.at[i,date_name] = y_pred_one[0][0][0]# adding this forecast to the dataframe
            
            # formatting the new_row
            new_row = meter_X_preds[j].copy()
            new_row[24]=y_pred_one[0][0][0]
            new_row = new_row.reshape((1,25,1)).astype('float32')
            new_row[np.isnan(new_row)] = 0
            
            
            #adding the n-k values in the subsequent k days
            for k in range(1,6):
                print(j)
                if(j+k>365):
                    meter_X_preds[j+k][24-k]=y_pred_one[0][0][0]
                else:
                    break
            
            print(new_row.shape)
            # print(new_row)
            print(meter_X.shape)
            
            meter_X = np.concatenate([meter_X, new_row], axis=0) #adding this prediction to the end of the X's
            
            j+=1

make_preds()

  0%|          | 0/3248 [00:00<?, ?it/s]

  0%|          | 0/365 [00:00<?, ?it/s]

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [ ]:
df_forecasts

## Conclusion

* far too slow and impractical
    * 17 hours to make predictions
    * MINIMUM of 2 minutes per epoch of training (if a model with 1 neuron and 1 layer)
    * With multiple layers this can go up to several minutes per epoch of training
    * Suitable amount of time to train the model would be a minimum of 1000 epochs (probably several thousand epochs)
    * Thus given the time constraint of the competition and the deadline this is not possible to explore further
    * HOWEVER, all the code has been done for the training and predictions of the model so as an extension if wished to further explore and had the computing power / time available to do so it would be worth experimenting to see what performance RNN based models (LSTM, GRU, etc) could achieve
    
    
* Currently model seems to just predict same value for all entries
* Probably due to simple topology and lack of epochs 

* For the reasons above going to use the LGBM instead